# 0 - Library imports

In [ ]:
# Builtins
import os
import sys
import gc
from pathlib import Path
# Installed
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
# Types
from typing import (
    Tuple,
    Any
)

In [2]:
# Set path for custom imports
project_root = os.path.abspath('..')
print(project_root)
sys.path.append(project_root)

/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25


In [ ]:
# Local
from src.dataloader import TorchDataLoader
from src.unet import (
    UNet3D,
    AttentionUnet3D
)
from src.vit import (
    VisionTransformer
)
from src.callbacks.logger import (
    LogFileLogger,
    CSVLogger
)
from src.callbacks.earlystopping import EarlyStopping
from src.callbacks.modelcheckpoint import ModelCheckpoint
from src.callbacks.reducelr import ReduceLROnPlateau
from src.train_logic import train_model

In [5]:
print(
    f"PyTorch detects {torch.cuda.device_count()} GPU(s): {', '.join(torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count()))}" 
    if torch.cuda.is_available() 
    else "PyTorch can't find any GPU"
)

PyTorch detects 1 GPU(s): NVIDIA A100-SXM4-80GB


# 1 - Global

## 1.1 - Vars

In [ ]:
BOLD = "\033[1m"      # Bold
ITALIC = "\033[3m"    # Italic
RESET = "\033[0m"     # Reset stdout modifications

DATA_DIR_PATH = Path('/home/promise/NAS/MATERIA_OSCURA/PROYECTOS_INVESTIGACION/2025-Direct_AC/001-Dataset/002-PET_NAC_AC/')

IMAGE_CUT_OPTIONS = {
    1: 'axial',
    2: 'coronal',
    3: 'sagittal'
}

MODEL_SAVE_OPTIONS = {
    1: 'state_dict',
    2: 'complete',
    3: 'with_params'
}

MODEL_TYPE = {
    1: 'unet-original',
    2: 'unet-attention',
    3: 'vit'
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1.2 - Functions

In [ ]:
def display_configurations(model_config: dict[str, Any], data_config: dict[str, Any]) -> None:
    """Show data and model configurations"""
    print("=== Settings ===")
    print(f"{BOLD} - Data Directory Path:{RESET:>5} {ITALIC}{DATA_DIR_PATH}{RESET}")
    print(f"{BOLD} - Using device:{RESET:>11} {ITALIC} {DEVICE}{RESET}")
    
    print("\n=== Data Configuration ===")
    max_data_key_len = max(len(key) for key in data_config.keys())
    for key, value in data_config.items():
        print(f"{BOLD} - {key + ':':<{max_data_key_len + 2}}{RESET} {ITALIC}{value}{RESET}")
    
    print("\n=== Model Configuration ===")
    max_model_key_len = max(len(key) for key in model_config.keys())
    for key, value in model_config.items():
        print(f"{BOLD} - {key + ':':<{max_model_key_len + 2}}{RESET} {ITALIC}{value}{RESET}")

In [ ]:
def load_data(model_config: dict[str, Any], data_config: dict[str, Any]) -> Tuple[DataLoader, DataLoader]:
    dataloader = TorchDataLoader(
        data_dir_path=DATA_DIR_PATH,
        data_config=data_config,
        model_config=model_config
    )

    return dataloader.load_train_data(), dataloader.load_validation_data()

In [ ]:
def create_and_train_model(model_config: dict[str, Any], train_loader: DataLoader, val_loader: DataLoader) -> dict[str, Any]:
    # Create model
    if model_config['model_type'] == 'unet-original':
        model = UNet3D(model_config)
    elif model_config['model_type'] == 'unet-attention':
        model = AttentionUnet3D(model_config)
    elif model_config['model_type'] == 'vit':
        model = VisionTransformer(model_config)
    else:
        raise ValueError(f"Unknown model type: {model_config['model_type']}")

    model.display_summary()

    if model_config.get('web_view', False):
        model.view_model_graph(browser_view=True)

    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=model_config['learning_rate'])
    criterion = nn.L1Loss()  # Mean Absolute Error (MAE)

    training_logger = LogFileLogger(model_config=model_config)

    # Setup callbacks
    callbacks = [
        ModelCheckpoint(
            model_config=model_config,
            verbose=True,
            save_best_only=True,
            logger=training_logger
        ),
        ReduceLROnPlateau(
            optimizer,
            model_config=model_config,
            logger=training_logger
        ),
        EarlyStopping(
            model_config=model_config,
            restore_best_weights=False,
            logger=training_logger
        ),
        CSVLogger(
            model_config=model_config,
            logger=training_logger
        ),
        training_logger
    ]

    # Train the model
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        criterion=criterion,
        device=DEVICE,
        config=model_config,
        callbacks=callbacks
    )
 
    return history

# **AVISO**

Los logs de entrenamiento de la U-net original no están en este notebook porque la celda se volvió a ejecutar por error.

Los logs de entrenamiento del ViT no se ven completos en este notebook.

Ambos logs pueden encontrarse respectivamente en la carpeta de cada modelo:

- **U-net**: *lab/models/Unet1/logs/model_train.log*

- **ViT**: *lab/models/vit_pruebas_2/logs/model_train.log*

# 2 - Normal U-Net

In [10]:
model_name = 'Unet1'
batch_size = 8
layer_qty = 8
data_config = {
    'axis_cut': IMAGE_CUT_OPTIONS[1],
    'first_cut': 0,
    'last_cut': 127,
    'layer_qty': layer_qty,
    'normalize': False
}
model_config = {
    'name': model_name,
    'model_type': MODEL_TYPE[1],
    'input_shape': (1, layer_qty, 128, 128),
    'input_shape_graph': (batch_size, 1, layer_qty, 128, 128),
    'batch_size': batch_size,
    'min_feature_map_dim': 8,
    'layer_filters': [64, 128, 256, 512, 1024],
    'layer_dropouts': [0.1, 0.1, 0.2, 0.2, 0.3],
    'num_epochs': 200,
    'learning_rate': 1e-4,
    'early_stopping_patience': 20,
    'reduce_lr_patience': 5,
    'lr_reduce_factor': 0.1,
    'min_lr': 1e-12,
    'save_method': MODEL_SAVE_OPTIONS[2],
    'web_view': True
}

display_configurations(model_config, data_config)

=== Settings ===
 - Data Directory Path:  /home/promise/NAS/MATERIA_OSCURA/PROYECTOS_INVESTIGACION/2025-Direct_AC/001-Dataset/002-PET_NAC_AC
 - Using device:         cuda

=== Data Configuration ===
 - axis_cut:   axial
 - first_cut:  0
 - last_cut:   127
 - layer_qty:  8
 - normalize:  False

=== Model Configuration ===
 - name:                     Unet1
 - model_type:               unet-original
 - input_shape:              (1, 8, 128, 128)
 - input_shape_graph:        (8, 1, 8, 128, 128)
 - batch_size:               8
 - min_feature_map_dim:      8
 - layer_filters:            [64, 128, 256, 512, 1024]
 - layer_dropouts:           [0.1, 0.1, 0.2, 0.2, 0.3]
 - num_epochs:               200
 - learning_rate:            0.0001
 - early_stopping_patience:  20
 - reduce_lr_patience:       5
 - lr_reduce_factor:         0.1
 - min_lr:                   1e-12
 - save_method:              complete
 - web_view:                 True


In [11]:
train_loader, val_loader = load_data(model_config, data_config)

history = create_and_train_model(model_config, train_loader, val_loader)

# Clean environment - Delete vars and free GPU memory (there is still something takin like 6% of the GPU V-RAM)
del history
del model_name
del batch_size
del data_config
del model_config
del train_loader
del val_loader
torch.cuda.empty_cache()
gc.collect()

Configuration saved to: '/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/Unet1/config/data_config.json'...
Configuration saved to: '/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/Unet1/config/model_config.json'...
Loading training data...
 ==  Loading train images set  ==
	- Loading 28 images from '/home/promise/NAS/MATERIA_OSCURA/PROYECTOS_INVESTIGACION/2025-Direct_AC/001-Dataset/002-PET_NAC_AC/fold1' ...


KeyboardInterrupt: 

# 3 - U-Net Attention

In [ ]:
model_name = 'attention1'
batch_size = 8
layer_qty = 8
data_config = {
    'axis_cut': IMAGE_CUT_OPTIONS[1],
    'first_cut': 0,
    'last_cut': 127,
    'layer_qty': layer_qty,
    'normalize': False
}
model_config = {
    'name': model_name,
    'model_type': MODEL_TYPE[2],
    'input_shape': (1, layer_qty, 128, 128),
    'input_shape_graph': (batch_size, 1, layer_qty, 128, 128),
    'batch_size': batch_size,
    'min_feature_map_dim': 8,
    'layer_filters': [64, 128, 256, 512, 1024],
    'layer_dropouts': [0.1, 0.1, 0.2, 0.2, 0.3],
    'num_epochs': 200,
    'learning_rate': 1e-4,
    'early_stopping_patience': 20,
    'reduce_lr_patience': 5,
    'lr_reduce_factor': 0.1,
    'min_lr': 1e-12,
    'save_method': MODEL_SAVE_OPTIONS[2],
    'web_view': True
}

display_configurations(model_config, data_config)

=== Settings ===
 - Data Directory Path:  /home/promise/NAS/MATERIA_OSCURA/PROYECTOS_INVESTIGACION/2025-Direct_AC/001-Dataset/002-PET_NAC_AC
 - Using device:         cuda

=== Data Configuration ===
 - axis_cut:   axial
 - first_cut:  0
 - last_cut:   127
 - layer_qty:  8
 - normalize:  False

=== Model Configuration ===
 - name:                     attention1
 - model_type:               unet-attention
 - input_shape:              (1, 8, 128, 128)
 - input_shape_graph:        (8, 1, 8, 128, 128)
 - batch_size:               8
 - min_feature_map_dim:      8
 - layer_filters:            [64, 128, 256, 512, 1024]
 - layer_dropouts:           [0.1, 0.1, 0.2, 0.2, 0.3]
 - num_epochs:               200
 - learning_rate:            0.0001
 - early_stopping_patience:  20
 - reduce_lr_patience:       5
 - lr_reduce_factor:         0.1
 - min_lr:                   1e-12
 - save_method:              complete
 - web_view:                 True


In [24]:
train_loader, val_loader = load_data(model_config, data_config)

history = create_and_train_model(model_config, train_loader, val_loader)

# Clean environment - Delete vars and free GPU memory (there is still something takin like 6% of the GPU V-RAM)
del history
del model_name
del batch_size
del data_config
del model_config
del train_loader
del val_loader
torch.cuda.empty_cache()
gc.collect()

Configuration saved to: '/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/attention1/config/data_config.json'...
Configuration saved to: '/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/attention1/config/model_config.json'...
Loading training data...
 ==  Loading train images set  ==
	- Loading 28 images from '/home/promise/NAS/MATERIA_OSCURA/PROYECTOS_INVESTIGACION/2025-Direct_AC/001-Dataset/002-PET_NAC_AC/fold1' ...
	- Could only load 28 images. Detected 0 outlier(s)
	- Pixels value range: [0.0, 0.22]
Train inputs shape: (448, 1, 8, 128, 128)
Train targets shape: (448, 1, 8, 128, 128)
Training loader generated. Batch size: 8
== Train set loader ==
Common batch shape (num_batches / batch_size, channels, depth / num_layers, height, width):
 - Inputs: torch.Size([8, 1, 8, 128, 128])
 - Targets: torch.Size([8, 1, 8, 128, 128])
All batches have the same size
Loading test data for validation...
 ==  Loading test images set  ==
	- Loading 8 images from '/home/prom

/usr/bin/xdg-open: 882: /home/promise/.vscode-server/cli/servers/Stable-17baf841131aa23349f217ca7c570c76ee87b957/server/bin/helpers/browser.sh: not found
/usr/bin/xdg-open: 882: /home/promise/.vscode-server/cli/servers/Stable-17baf841131aa23349f217ca7c570c76ee87b957/server/bin/helpers/browser.sh: not found
xdg-open: no method available for opening '/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/attention1/graph/attention1.pdf'
Epoch 1/200: 100%|██████████| 56/56 [00:11<00:00,  4.81it/s, loss=0.265, mae=0.265, lr=0.0001]


Epoch 1/200 - 12.70s - loss: 0.2655 - mae: 0.2655 - val_loss: 0.2054 - val_mae: 0.2054 - learning_rate: 0.0001

Epoch 1: val_loss improved from inf to 0.20539, saving model to /home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/attention1/models/attention1.pth


Epoch 2/200: 100%|██████████| 56/56 [00:11<00:00,  4.82it/s, loss=0.201, mae=0.201, lr=0.0001]


Epoch 2/200 - 12.70s - loss: 0.2008 - mae: 0.2008 - val_loss: 0.1855 - val_mae: 0.1855 - learning_rate: 0.0001

Epoch 2: val_loss improved from 0.20539 to 0.18549, saving model to /home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/attention1/models/attention1.pth


...

Epoch 131/200: 100%|██████████| 56/56 [00:11<00:00,  4.84it/s, loss=0.00488, mae=0.00488, lr=1e-11]


Epoch 131/200 - 12.63s - loss: 0.0049 - mae: 0.0049 - val_loss: 0.0109 - val_mae: 0.0109 - learning_rate: 1.0000000000000006e-11


Epoch 132/200: 100%|██████████| 56/56 [00:11<00:00,  4.84it/s, loss=0.00479, mae=0.00479, lr=1e-11]
Exception ignored in: <function LogFileLogger.__del__ at 0x7f5bc09367a0>
Traceback (most recent call last):
  File "/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/src/callbacks/logger.py", line 96, in __del__
    self.close()  # Ensure the file is closed when the object is deleted
  File "/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/src/callbacks/logger.py", line 92, in close
    self.file.write(end_message)
ValueError: I/O operation on closed file.


Epoch 132/200 - 12.66s - loss: 0.0048 - mae: 0.0048 - val_loss: 0.0112 - val_mae: 0.0112 - learning_rate: 1.0000000000000006e-11
Early stopping triggered


5779

# 4 - ViT

In [ ]:
model_name = 'vit_pruebas_2'
batch_size = 8
layer_qty = 8
data_config = {
    'axis_cut': IMAGE_CUT_OPTIONS[1],
    'first_cut': 0,
    'last_cut': 127,
    'layer_qty': layer_qty,
    'normalize': False
}

model_config = {
    "name": model_name,
    'model_type': MODEL_TYPE[3],
    "input_shape": [1, layer_qty, 128, 128],  # (..., channels, height, width)
    "input_shape_graph": [batch_size, 1, layer_qty, 128, 128],  # (..., channels, height, width)
    "output_channels": 1,  # Same as input channels
    "batch_size": batch_size,
    "patch_size": 16,
    "embed_dim": 768,
    "num_heads": 12,
    "num_layers": 8,
    "dropout": 0.15,
    "mlp_ratio": 2.0,
    "layer_dropouts": [0.1, 0.1, 0.12, 0.12, 0.15, 0.15, 0.18, 0.18],
    "overlap_ratio": 0.5,            # 50% overlap between patches
    "use_spatial_attention": True,   # Enable spatial-aware transformer blocks
    "use_smoothing": True,           # Enable post-processing smoothing
    'num_epochs': 200,
    'learning_rate': 1e-4,
    'early_stopping_patience': 20,
    'reduce_lr_patience': 4,
    'lr_reduce_factor': 0.1,
    'min_lr': 1e-12,
    'save_method': MODEL_SAVE_OPTIONS[2],
    'web_view': False
}

display_configurations(model_config, data_config)

=== Settings ===
 - Data Directory Path:  /home/promise/NAS/MATERIA_OSCURA/PROYECTOS_INVESTIGACION/2025-Direct_AC/001-Dataset/002-PET_NAC_AC
 - Using device:         cuda

=== Data Configuration ===
 - axis_cut:   axial
 - first_cut:  0
 - last_cut:   127
 - layer_qty:  8
 - normalize:  False

=== Model Configuration ===
 - name:                     vit_pruebas_2
 - model_type:               vit
 - input_shape:              [1, 8, 128, 128]
 - input_shape_graph:        [8, 1, 8, 128, 128]
 - output_channels:          1
 - batch_size:               8
 - patch_size:               16
 - embed_dim:                768
 - num_heads:                12
 - num_layers:               8
 - dropout:                  0.15
 - mlp_ratio:                2.0
 - layer_dropouts:           [0.1, 0.1, 0.12, 0.12, 0.15, 0.15, 0.18, 0.18]
 - overlap_ratio:            0.5
 - use_spatial_attention:    True
 - use_smoothing:            True
 - num_epochs:               200
 - learning_rate:            0.0001
 - 

In [ ]:
train_loader, val_loader = load_data(model_config, data_config)

history = create_and_train_model(model_config, train_loader, val_loader)

# Clean environment - Delete vars and free GPU memory (there is still something takin like 6% of the GPU V-RAM)
del history
del model_name
del batch_size
del data_config
del model_config
del train_loader
del val_loader
torch.cuda.empty_cache()
gc.collect()

Configuration saved to: '/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/vit_pruebas_2/config/data_config.json'...
Configuration saved to: '/home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/vit_pruebas_2/config/model_config.json'...
Loading training data...
 ==  Loading train images set  ==
	- Loading 28 images from '/home/promise/NAS/MATERIA_OSCURA/PROYECTOS_INVESTIGACION/2025-Direct_AC/001-Dataset/002-PET_NAC_AC/fold1' ...
	- Could only load 28 images. Detected 0 outlier(s)
	- Pixels value range: [0.0, 0.22]
Train inputs shape: (448, 1, 8, 128, 128)
Train targets shape: (448, 1, 8, 128, 128)
Training loader generated. Batch size: 8
== Train set loader ==
Common batch shape (num_batches / batch_size, channels, depth / num_layers, height, width):
 - Inputs: torch.Size([8, 1, 8, 128, 128])
 - Targets: torch.Size([8, 1, 8, 128, 128])
All batches have the same size
Loading test data for validation...
 ==  Loading test images set  ==
	- Loading 8 images from '/hom

Epoch 1/200: 100%|██████████| 56/56 [04:13<00:00,  4.52s/it, loss=0.0551, mae=0.0551, lr=0.0001]


Epoch 1/200 - 257.69s - loss: 0.0551 - mae: 0.0551 - val_loss: 0.0406 - val_mae: 0.0406 - learning_rate: 0.0001

Epoch 1: val_loss improved from inf to 0.04058, saving model to /home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/vit_pruebas_2/models/vit_pruebas_2.pth


Epoch 2/200: 100%|██████████| 56/56 [04:12<00:00,  4.52s/it, loss=0.0304, mae=0.0304, lr=0.0001]


Epoch 2/200 - 257.33s - loss: 0.0304 - mae: 0.0304 - val_loss: 0.0235 - val_mae: 0.0235 - learning_rate: 0.0001

Epoch 2: val_loss improved from 0.04058 to 0.02355, saving model to /home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/vit_pruebas_2/models/vit_pruebas_2.pth


...

Epoch 32/200: 100%|██████████| 56/56 [04:12<00:00,  4.52s/it, loss=0.00763, mae=0.00763, lr=1e-5]


Epoch 32/200 - 257.26s - loss: 0.0076 - mae: 0.0076 - val_loss: 0.0134 - val_mae: 0.0134 - learning_rate: 1e-05

Epoch 32: val_loss improved from 0.01434 to 0.01335, saving model to /home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/vit_pruebas_2/models/vit_pruebas_2.pth


Epoch 33/200: 100%|██████████| 56/56 [04:12<00:00,  4.51s/it, loss=0.00739, mae=0.00739, lr=1e-5]


Epoch 33/200 - 256.69s - loss: 0.0074 - mae: 0.0074 - val_loss: 0.0133 - val_mae: 0.0133 - learning_rate: 1e-05

Epoch 33: val_loss improved from 0.01335 to 0.01325, saving model to /home/promise/jhernandez/3d-notebooks/3d/9jun25/9jun25/models/vit_pruebas_2/models/vit_pruebas_2.pth


Epoch 34/200:  80%|████████  | 45/56 [03:22<00:49,  4.52s/it, loss=0.00732, mae=0.00732, lr=1e-5]